In [1]:
import os
os.environ["SPS_HOME"] = "/Users/fpetri/packages/fsps" 

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import lbg_forecast.sps as sps
import lbg_forecast.sfh as sfh
import lbg_forecast.priors as pr
import lbg_forecast.hyperparams as hyp
import lbg_forecast.popmodel as pop
import lbg_forecast.zhistory as zh
import lbg_forecast.priors as pr
from astropy.cosmology import WMAP9 as cosmo
from scipy.stats import truncnorm

In [3]:
sps_model = sps.initialise_sps_model(neb_em=False)

In [4]:
prior_params = pr.setup_redshift_and_mass_priors(z_max=15)

In [8]:
def truncated_normal(mu, sigma, min, max, samples):
    a, b = (min - mu) / sigma, (max - mu) / sigma
    return truncnorm.rvs(a, b, loc=mu, scale=sigma, size=samples)

In [9]:
def generate_sps_parameters(nsamples, prior_params):

    logzsol_mu, logzsol_sigma = -0.1, 0.1
    dust1_mu, dust1_sigma = 0.2, 0.5
    dust2_mu, dust2_sigma = 0.2, 0.5
    dust_index_mu, dust_index_sigma = -0.7, 0.4

    #Setup redshift and mass priors
    z_grid, logm_grid, priors, grid_params = prior_params
    z_samples, m_samples = pr.sample_priors(z_grid, logm_grid, priors, grid_params, nsamples, plotting=False)
    
    #Ages of universe at different redshifts
    t_universe_grid = pop.tuniv_grid(z_grid)    

    nsps_parameters = []

    #Redshift - zred
    redshift = np.vstack(z_samples)
    nsps_parameters.append(redshift)

    #Age of the universe at given redshift - tage [Gyr]
    tuniv = np.interp(z_samples, z_grid, t_universe_grid)
    tuniv = np.reshape(np.log10(np.asarray(tuniv)), (nsamples,))
    nsps_parameters.append(tuniv)

    #Stellar Metallicity - logzsol
    logzsol_min = -2.5
    logzsol_max = 0.5
    logzsol = truncated_normal(logzsol_mu, logzsol_sigma, logzsol_min, logzsol_max)
    nsps_parameters.append(logzsol)

    #Dust parameter for attenuating young starlight - dust1
    dust1_min = 0.0
    dust1_max = 2.0
    dust1 = truncated_normal(dust1_mu, dust1_sigma, dust1_min, dust1_max) 
    nsps_parameters.append(dust1)

    #Diffuse dust parameter - dust2
    dust2_min = 0.0
    dust2_max = 4.0
    dust2 = truncated_normal(dust2_mu, dust2_sigma, dust2_min, dust2_max) 
    nsps_parameters.append(dust2)

    #Index of dust attenuation law - dust_index
    dust_index_min = -1.0
    dust_index_max = 0.4
    dust_index = truncated_normal(dust_index_mu, dust_index_sigma, dust_index_min, dust_index_max) 
    nsps_parameters.append(dust_index)

    return nsps_parameters
